<a href="https://colab.research.google.com/github/Bibhash123/Project-Primary-Quantization/blob/main/experiments/primary_quantization_attention_net_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
COLAB = False
if COLAB:
    DATA_DIR = "/content/Data"
else:
    DATA_DIR = "../input/tondidataset/"

In [ ]:
if COLAB:
    from google.colab import files
    _ = files.upload()
    !mkdir ~/.kaggle/
    !cp kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    !pip install -q kaggle
    !kaggle datasets download -d "bibhash123/tondidataset"
    !unzip -q tondidataset.zip -d "/content/Data/"
    !rm -r tondidataset.zip

In [ ]:
import os
import gc
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.losses import categorical_crossentropy, mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from functools import partial

In [ ]:
train_files = pd.read_csv(os.path.join(DATA_DIR, "mini_train.csv"),sep=";",header=None,
                         names=['idx', 'filenames', 'quality1', 'quality2', 'software', 'labels',
                                'shift_r', 'shift_c','base_image']
                         )
trn,val = train_test_split(train_files,test_size=0.1)

In [ ]:
print("Train Shape: ", trn.shape)
print("Validation Shape: ", val.shape)

Train Shape:  (97200, 9)
Validation Shape:  (10800, 9)


In [ ]:
class cfg:
    def __init__(self):
        pass
    
    coeffs_low = [13,9,10,11,10,8,13,11,10,11,14,14,13,15,19,32,21,19,18,18,19,39,28,30,23,32,46,41,49,48,46,41,45,44,51,58,74,62,51,54,70,55,44,45,64,87,65,70,76,78,82,83,82,50,62,90,97,90,80,96,74,81,82,79]
    max_vals = coeffs_low[:15]
    num_outputs = np.sum(max_vals)

In [ ]:
def preprocess_input(im_file, target_size, scale=255.):
    """ 
        Read image and (eventually) scale data
        Arguments:
            im_file     : input image file
            target_size : output size of the image (height, width)
            scale       : pixel scaling value
        Returns: The image
    """
    file_bytes = tf.io.read_file(im_file)
    img = tf.image.decode_png(file_bytes, channels = 0)
    # Normalize and Resize
    if img.shape != target_size:
        img = tf.image.resize(img, target_size)
    img = tf.cast(img/scale, tf.float32)
    return img
  
def string2Q(s, size=(8, 8), flatten=True):
    """ Converts a comma separated string to a matrix.
        Keyword arguments:
        sq : input string
        size : output matrix size
    """
    if flatten:
        return tf.strings.to_number(tf.strings.split(s,','),out_type=tf.int32)
    else:
        return tf.reshape(tf.strings.to_number(tf.strings.split(s,','),out_type=tf.int32),size)
        
def get_label(im_label):
    labs = string2Q(im_label)[:15]
    labels = []
    for i in range(15):
        t = tf.one_hot([labs[i]-1],cfg.max_vals[i])
        labels.append(t)
    return tf.reshape(K.concatenate(labels,axis=-1),[-1])

def getQFRange(qf1):
    return K.stack([qf1-5,qf1,qf1+5],axis=0)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def build_decoder(is_labelled):
    def if_labelled(path,label, target_size):
        image = preprocess_input(path,target_size)
        out_img = K.concatenate([tf.expand_dims(tf.image.adjust_jpeg_quality(image,qf),axis=0) for qf in [60,70,80,90,98]],axis=0)
        label = get_label(label)
        # qf = getQFRange(qf1)
        return (image, out_img), label
  
    def not_labelled(path,target_size):
        image = preprocess_input(path,target_size)
        out_img = K.concatenate([tf.expand_dims(tf.image.adjust_jpeg_quality(image,qf),axis=0) for qf in [60,70,80,90,98]],axis=0)
        return (image, out_img)    

    return if_labelled if is_labelled else not_labelled


def create_dataset(df, batch_size = 32, is_labelled = False, repeat = False, shuffle = False, batch=False, cache=False):
    decode_fn = build_decoder(is_labelled)
    df['filenames'] = df['filenames'].apply(lambda x: "/".join(x.split('/')[-3:]))
    df['filenames'] = df['filenames'].apply(lambda x: os.path.join(DATA_DIR,x))
    
    # Create Dataset
    if is_labelled:
        dataset = tf.data.Dataset.from_tensor_slices((df['filenames'].values,df["labels"].values))
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df['filenames'].values))

    dataset = dataset.map(partial(decode_fn,target_size=(64,64)), num_parallel_calls = AUTOTUNE)
    dataset = dataset.cache("") if cache else dataset
    dataset = dataset.repeat() if repeat else dataset
    dataset = dataset.shuffle(1024, reshuffle_each_iteration = True) if shuffle else dataset
    dataset = dataset.batch(batch_size,drop_remainder=True) if batch else dataset
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

def getX(X,Y):
    return X
def getY(X,Y):
    return Y

In [ ]:
def custom_softmax_activation(max_vals):

    """
    Params:
    =======================================
    max_vals: qm values for i from 1 to 15
    """
    bins = np.concatenate(([0], np.cumsum(max_vals))).astype(np.int16)
    def custom_softmax(x):
        stack_list = [softmax(x[:, int(bins[i]):int(bins[i + 1])]) for i in range(0, len(bins)-1)]
        return tf.concat(stack_list, axis=-1)

    return custom_softmax

def custom_mse_wrapper(max_vals):

    """
    Keras custom "piece-wise" MSE metric
    :param max_vals: bins for piecewise computation
    :return: custom mse function
    """
    bins = np.concatenate(([0], np.cumsum(max_vals))).astype(np.int)
    def custom_mse(y_true, y_pred):

        mse_val = tf.constant(0, dtype=tf.float64)

        for i in range(0, len(bins) - 1):

            y_true_to_coeff = tf.argmax(y_true[:, bins[i]:bins[i+1]], 1) + 1
            y_pred_to_coeff = tf.argmax(y_pred[:, bins[i]:bins[i+1]], 1) + 1

            mse_val = tf.add(mse_val,
                             tf.reduce_sum(tf.square(y_true_to_coeff - y_pred_to_coeff)) / cfg.batch_size)
        return mse_val

    return custom_mse


# Trick to pass parameters to custom losses
def custom_categorical(max_vals):
    bins = np.concatenate(([0], np.cumsum(max_vals))).astype(np.int16)
    def custom_categorical_crossentropy(y_true, y_pred):

        """ Keras custom loss for QF Estimation with the new labeling strategy.
        Now the label is a categorical array with values [0, 1]. The length of the
        array is equal to the sum of values in max_vals, that is the maximum value
        each coefficient can assume. Index == value is set to 1. E.g., for 2 coefficients
        [6, 7] where max_length is [11, 12]:
            y_true = [0 0 0 0 0 1 0 0 0 0 0, 0 0 0 0 0 0 1 0 0 0 0 0]
        Categorical crossentropy is computed on each individual sub-label rather than on full y_true.
        Each "chunk" of loss is added to total loss
        Example:
        loss(y_true[:, 0:11], y_pred[:, 0:11]) + loss(y_true[:, 11:], y_pred[:, 11:])
        :param y_true: Tensorflow/Theano tensor of predicted labels
        :param y_pred: Tensorflow/Theano tensor of true labels
        :return: Custom loss
        """

        loss = categorical_crossentropy(y_true[:, int(bins[0]):int(bins[1])],
                                        y_pred[:, int(bins[0]):int(bins[1])])
        for i in range(1, len(bins) - 1):
            loss = tf.add(loss, categorical_crossentropy(y_true[:, int(bins[i]):int(bins[i + 1])],
                                                         y_pred[:, int(bins[i]):int(bins[i + 1])]))
        return loss

    return custom_categorical_crossentropy

In [ ]:
train_set = create_dataset(trn, batch_size = 32, is_labelled = True, repeat = True, 
                          shuffle = True, batch=True,cache=False)
val_set = create_dataset(val, batch_size = 32, is_labelled = True, repeat = False, 
                          shuffle = False, batch=True,cache=False)
Y_val = val_set.map(getY)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
2022-04-05 18:13:11.486128: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-05 18:13:11.628720: I tensorflow/stream_executor/cuda/cuda_gpu_exe

In [ ]:
train_set.take(1)

<TakeDataset shapes: (((32, 64, 64, None), (32, 5, None, None, None)), (32, 181)), types: ((tf.float32, tf.float32), tf.float32)>

In [ ]:
class CrossAttention(L.Layer):
    def __init__(self,input_dim, hidden_dim, output_dim):
        super(CrossAttention,self).__init__()
        self.w_k = self.add_weight(name='key', shape = (input_dim,hidden_dim), initializer="random_normal", trainable=True)
        self.w_q = self.add_weight(name='query', shape = (input_dim,hidden_dim), initializer="random_normal", trainable=True)
        self.w_v = self.add_weight(name='value', shape = (input_dim,output_dim), initializer="random_normal", trainable=True)

    def call(self,X1,X2):
        dims = 5
        key = tf.expand_dims(tf.matmul(X1,self.w_k),axis=1)
        query = K.concatenate([tf.expand_dims(tf.matmul(X2[:,i,:],self.w_q),axis=2) for i in range(dims)],axis=2)
        value = K.concatenate([tf.expand_dims(tf.matmul(X2[:,i,:],self.w_v),axis=2) for i in range(dims)],axis=2)
        score = tf.matmul(tf.nn.softmax(tf.matmul(key,query),axis=-1),tf.transpose(value,perm=[0,2,1]))
        return score[:,0,:]

In [ ]:
from densenet import DenseNet, ConvConst
K.clear_session()
with tf.device('/GPU:0'):
    err_model,_ = DenseNet(input_shape = (64,64,1), nb_classes = 15, depth=18)
    err_model = tf.keras.Model(inputs=err_model.input, outputs = err_model.layers[-2].output)
#     err_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights=None, input_shape=(64,64,1))
    img_model,_ = DenseNet(input_shape = (64,64,1), nb_classes = 15, depth=18)
    img_model = tf.keras.Model(inputs=img_model.input, outputs = img_model.layers[-2].output)
#     img_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights=None, input_shape=(64,64,1))

    inp1 = L.Input(shape=(64,64,1))
    inp2 = L.Input(shape=(5,64,64,1))
    h1 = img_model(inp1)
    h2 = L.TimeDistributed(err_model)(inp2)
    h2 = L.TimeDistributed(L.Flatten())(h2)
    h2 = L.TimeDistributed(L.Dense(128,activation='relu'))(h2)
    h1 = L.Flatten()(h1)
    h1 = L.Dense(128,activation='relu')(h1)
    
    h = CrossAttention(128,128,128)(h1,h2)
    h = L.Dense(cfg.num_outputs,activation= custom_softmax_activation(cfg.max_vals))(h)
    model = tf.keras.Model(inputs = [inp1,inp2],
                         outputs = h)
#     schedule = tf.keras.optimizers.schedules.ExponentialDecay(1e-4,
#                                                               decay_steps=3037,
#                                                               decay_rate=0.96,
#                                                               staircase=True)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)
    model.compile(loss = custom_categorical(cfg.max_vals), optimizer = opt, metrics=  ['accuracy'])
    model.summary()

Creating DenseNet
#############################################
Dense blocks: 3
Layers per dense block: [4, 4, 4]
#############################################
Creating DenseNet
#############################################
Dense blocks: 3
Layers per dense block: [4, 4, 4]
#############################################
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 5, 64, 64, 1 0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, 168)          142200      inpu

In [ ]:
class Logger(tf.keras.callbacks.Callback):
    def on_epoch_begin(self,epoch,logs={}):
        print(" Learning Rate: {}".format(self.model.optimizer._decayed_lr(tf.float32).numpy()))

In [ ]:
class ConstWeight(tf.keras.callbacks.Callback):
    def on_batch_end(self,epoch,logs={}):
        weights = self.model.layers[2].layers[0].get_weights()
        weights[0][2,2,0,:] = np.zeros((12,))
        for i in range(weights[0].shape[-1]):
            t = weights[0][:,:,0,i]
            nom = -1*np.sum(t)
            weights[0][:,:,:,i] = weights[0][:,:,:,i]/nom
        weights[0][2,2,0,:] = np.ones((12,))  
        self.model.layers[2].layers[0].set_weights(weights)

        weights = self.model.layers[3].model.layers[0].get_weights()
        weights[0][2,2,0,:] = np.zeros((12,))
        for i in range(weights[0].shape[-1]):
            t = weights[0][:,:,0,i]
            nom = -1*np.sum(t)
            weights[0][:,:,:,i] = weights[0][:,:,:,i]/nom
        weights[0][2,2,0,:] = np.ones((12,))  
        self.model.layers[3].model.layers[0].set_weights(weights)

In [ ]:
ckpt = tf.keras.callbacks.ModelCheckpoint('model.hdf5', monitor = 'val_loss', mode='min',
                                          save_best_only = True, save_weights_only = True)
es = tf.keras.callbacks.EarlyStopping(patience = 7, monitor = 'val_loss', mode='min',
                                      restore_best_weights=True)
logger = Logger()
# const = ConstWeight()
try:
    # model.load_weights('last_trained.hdf5')
    model.fit(train_set,
            epochs = 60,
            steps_per_epoch = (trn.shape[0]//32),
            validation_data = val_set,
            callbacks = [ckpt,es],
            initial_epoch=0
            )
except KeyboardInterrupt:
    print("\n[INFO] Interrupted Training")
    model.save_weights('last_trained.hdf5')
print('[INFO] Obtaining Predictions')
model.load_weights('model.hdf5')

preds = []
pred = model.predict(val_set)
bins = np.concatenate(([0], np.cumsum(cfg.max_vals))).astype(np.int)
for i in range(0, len(bins) - 1):
    y_true_to_coeff = tf.argmax(pred[:, bins[i]:bins[i+1]], 1) + 1
    preds.append(y_true_to_coeff)
preds = np.stack(preds,axis=1)

Epoch 1/60


2022-04-05 18:13:24.651987: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-05 18:13:34.205183: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


3037/3037 [==============================] - 617s 195ms/step - loss: 32.2185 - accuracy: 0.0000e+00 - val_loss: 31.9359 - val_accuracy: 0.0000e+00
Epoch 2/60
3037/3037 [==============================] - 519s 171ms/step - loss: 30.5416 - accuracy: 0.0023 - val_loss: 30.7367 - val_accuracy: 0.0032
Epoch 3/60
3037/3037 [==============================] - 519s 171ms/step - loss: 28.0725 - accuracy: 0.0048 - val_loss: 31.9391 - val_accuracy: 0.0191
Epoch 4/60
3037/3037 [==============================] - 519s 171ms/step - loss: 26.6756 - accuracy: 0.0074 - val_loss: 29.7580 - val_accuracy: 0.0096
Epoch 5/60
3037/3037 [==============================] - 521s 172ms/step - loss: 25.6867 - accuracy: 0.0097 - val_loss: 29.8900 - val_accuracy: 0.0176
Epoch 6/60
3037/3037 [==============================] - 529s 174ms/step - loss: 24.8278 - accuracy: 0.0111 - val_loss: 25.2304 - val_accuracy: 0.0114
Epoch 7/60
3037/3037 [==============================] - 520s 171ms/step - loss: 24.0685 - accuracy: 0.0

In [ ]:
y = np.array(list(Y_val.unbatch().as_numpy_iterator()))
y_true = []
for i in range(0, len(bins) - 1):
    y_true.append(tf.argmax(y[:, bins[i]:bins[i+1]], 1) + 1)
y_true = np.stack(y_true,axis=1)

In [ ]:
print("Shape Y: ",y_true.shape)
print("Accuracy = {:.4f}".format(np.sum(np.round(preds)==y_true)/(15*y_true.shape[0])))
print("MSE = {:.4f}".format(np.sum(np.square(y_true - preds)) / (y_true.shape[0]*15)))